## Data

In [1]:
# Recimo da ze imamo slike za training in validation
import os

train_dir_name = "../image_slicing/data/train/"
validation_dir_name = "../image_slicing/data/validation/"

train_positive_dir = os.path.join(train_dir_name + "positive")
train_negative_dir = os.path.join(train_dir_name + "negative")

validation_positive_dir = os.path.join(validation_dir_name + "positive")
validation_negative_dir = os.path.join(validation_dir_name + "negative")

In [2]:
# print total length of each dataset
print('total training positive images:', len(os.listdir(train_positive_dir)))
print('total training negative images:', len(os.listdir(train_negative_dir)))
print('total validation positive images:', len(os.listdir(validation_positive_dir)))
print('total validation negative images:', len(os.listdir(validation_negative_dir)))

total training positive images: 298
total training negative images: 105
total validation positive images: 85
total validation negative images: 30


In [10]:
# preprocessing needs to be done:
preprocessing_needed = True
img_shape = (350, 350, 3) #??
from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # All images will be rescaled by 1./255

if preprocessing_needed:
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir_name,
        target_size=(img_shape[0], img_shape[1]), # to be discussed?
        batch_size=30,
        class_mode='binary')
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir_name,
        target_size=(img_shape[0], img_shape[1]), # to be discussed?
        batch_size=10,
        class_mode='binary',
        shuffle=False)

Found 403 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


## Model

In [11]:
# imports
import tensorflow as tf
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [38]:


# model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = img_shape),
#                                     tf.keras.layers.Dense(128, activation=tf.nn.relu),
#                                     tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])
# print(model.summary())

model = tf.keras.models.Sequential([
# Note the input shape is the desired size of the image 200x200 with 3 bytes color
# This is the first convolution
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(350, 350, 3)),
tf.keras.layers.MaxPooling2D(2, 2),
# The second convolution
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# The third convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# The fourth convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# # The fifth convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# Flatten the results to feed into a DNN
tf.keras.layers.Flatten(),
# 512 neuron hidden layer
tf.keras.layers.Dense(512, activation='relu'),
# Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('dandelions') and 1 for the other ('grass')
tf.keras.layers.Dense(1, activation='sigmoid')])

In [39]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

TODO: Model checkpoints!! https://keras.io/api/callbacks/model_checkpoint/ 

In [40]:

# BEFORE:
# history = model.fit(train_generator,
#       steps_per_epoch=8,
#       epochs=15,
#       verbose=1,
#       validation_data = validation_generator,
#       validation_steps=8)

# AFTER:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

# Checkpoint directory
checkpoint_dir = "checkpoints/"

# Create directory if it does not exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_path = "checkpoints/model_{epoch:02d}-{val_loss:.2f}.keras"

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,  # Save only the best model
                             monitor='val_loss',   # Monitor the validation loss
                             mode='min',           # Minimize the validation loss
                             verbose=1)            # Print out when a model is being saved

In [41]:
history = model.fit(train_generator,
                    steps_per_epoch=8,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator,
                    validation_steps=8,
                    callbacks=[checkpoint]) 

Epoch 1/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7803 - loss: 0.7305
Epoch 1: val_loss improved from inf to 0.67227, saving model to checkpoints/model_01-0.67.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.7778 - loss: 0.7282 - val_accuracy: 0.6250 - val_loss: 0.6723
Epoch 2/15
6/8 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.6986 - loss: 0.6302

2024-03-26 20:28:14.302370: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 2: val_loss improved from 0.67227 to 0.16404, saving model to checkpoints/model_02-0.16.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7059 - loss: 0.6045 - val_accuracy: 1.0000 - val_loss: 0.1640
Epoch 3/15


2024-03-26 20:28:14.772144: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7546 - loss: 0.5707
Epoch 3: val_loss did not improve from 0.16404
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.7524 - loss: 0.5729 - val_accuracy: 0.6250 - val_loss: 0.6616
Epoch 4/15
6/8 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.7653 - loss: 0.5695

...

2024-03-26 20:29:21.980346: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 8: val_loss did not improve from 0.16404
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 991ms/step - accuracy: 0.7103 - loss: 0.5405 - val_accuracy: 1.0000 - val_loss: 0.2542
Epoch 9/15


2024-03-26 20:29:22.479962: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7387 - loss: 0.5526
Epoch 9: val_loss did not improve from 0.16404
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.7389 - loss: 0.5510 - val_accuracy: 0.6250 - val_loss: 0.6113
Epoch 10/15
3/8 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7796 - loss: 0.4688

KeyboardInterrupt: 

## Loading the model

In [43]:
# Load data/test/positive and data/test/negative and test the model on it
model_path = "checkpoints/model_02-0.16.keras"

model = tf.keras.models.load_model(model_path)

test_dir_name = "../image_slicing/data/test/"
test_positive_dir = os.path.join(test_dir_name + "positive")
test_negative_dir = os.path.join(test_dir_name + "negative")

print('total test positive images:', len(os.listdir(test_positive_dir)))
print('total test negative images:', len(os.listdir(test_negative_dir)))

test_positive_datagen = ImageDataGenerator(rescale=1./255)
test_negative_datagen = ImageDataGenerator(rescale=1./255)

test_positive_generator = test_positive_datagen.flow_from_directory(
    test_dir_name,
    target_size=(img_shape[0], img_shape[1]),
    batch_size=10,
    class_mode='binary',
    shuffle=False)

test_negative_generator = test_negative_datagen.flow_from_directory(
    test_dir_name,
    target_size=(img_shape[0], img_shape[1]),
    batch_size=10,
    class_mode='binary',
    shuffle=False)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_positive_generator)
print("test loss, test acc:", results)


total test positive images: 43
total test negative images: 16
Found 59 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Evaluate on test data
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.4863 - loss: 1.0431
test loss, test acc: [0.6492733359336853, 0.7288135886192322]
